In [1]:
import numpy as np
import h5py
import math
from sklearn.decomposition import PCA
from pca import pca
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import peakutils
import os, sys
from BaselineRemoval import BaselineRemoval
from sklearn.cluster import DBSCAN
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
class HiddenPrints:
	def __enter__(self):
		self._original_stdout = sys.stdout
		sys.stdout = open(os.devnull, 'w')

In [3]:
def __exit__(self, exc_type, exc_val, exc_tb):
	sys.stdout.close()
	sys.stdout = self._original_stdout

In [4]:
def remove_outliers(xset, yset, zset, eset, threshold):
	"""
	Uses DBSCAN to find and remove outliers in 3D data
	"""
	data = np.array([xset.T, yset.T, zset.T]).T
	DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
	out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)
	del data
	rev = out_of_cluster_index[0][::-1]
	#if len(out_of_cluster_index[0]) > 0:
	for i in rev:
		xset = np.delete(xset, i)
		yset = np.delete(yset, i)
		zset = np.delete(zset, i)
		eset = np.delete(eset, i)
	#if len(xset) <= threshold:
	#	veto = True
	#else:
    #veto = False
	return xset, yset, zset, eset, False

In [5]:
print('Veto on Length @ 70 mm')
print('Veto on Integrated Charge @ 800k')
def track_len(xset, yset, zset):
    """
    Uses PCA to find the length of a track
    """
    veto_on_length = False
 
    # Form data matrix
    data = np.concatenate((xset[:, np.newaxis], 
                           yset[:, np.newaxis], 
                           zset[:, np.newaxis]), 
                           axis=1)

    # Use PCA to find track length
    pca = PCA(n_components=3)
    principalComponents = pca.fit(data)
    principalComponents = pca.transform(data)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    
    track_len = 2.35*principalDf.std()[0]
    track_width = 2.35*principalDf.std()[1]
    track_depth = 2.35*principalDf.std()[2]
    #if track_len > 70:
    #    veto_on_length = True
    
    return track_len, veto_on_length, track_width, track_depth

Veto on Length @ 70 mm
Veto on Integrated Charge @ 800k


In [6]:
def main(h5file, threshold):
    """
    This functions does the following: 
    - Converts h5 files into ndarrays. 
    - Removes outliers.
    - Calls PCA to return track length.
    - Sums mesh signal to return energy.
    """
    # Converts h5 files into ndarrays, and output each event dataset as a separte list
    num_events = int(len(list(h5file.keys()))/2) 
    
    len_list = []
    width_list = []
    depth_list = []
    good_events = []
    tot_energy = []
    tracemax_list = []
    tracedev_list = []
    pads_list = []
    
    skipped_events = 0
    veto_events = 0
    
    pbar = tqdm(total=num_events+1)
    for i in range(0, num_events):
        str_event = f"Event_[{i}]"
        
        # Apply pad threshold
        event = np.array(h5file[str_event][:])
        if len(event) <= threshold:
            skipped_events += 1
            pbar.update(n=1)
            continue
            
        # Make copy of datasets
        dset_0_copyx = event['x']
        dset_0_copyy = event['y'] 
        dset_0_copyz = event['z'] - min(event['z'])
        dset_0_copye = event['A']

        
        # Apply veto condition
        R = 36                           # Radius of the pad plane
        r = np.sqrt(dset_0_copyx**2 + dset_0_copyy**2)
        statements = np.greater(r, R)    # Check if any point lies outside of R
      
        if np.any(statements) == True:
            veto_events += 1
            pbar.update(n=1)
            continue
        
        
        # Call remove_outliers to get dataset w/ outliers removed
        dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, veto = remove_outliers(dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, threshold)
        if veto == True:
            skipped_events += 1
            pbar.update(n=1)
            continue

        
        # Call track_len() to create lists of all track lengths
        length, veto_on_length, width, depth = track_len(dset_0_copyx, dset_0_copyy, dset_0_copyz)
        if veto_on_length == True:
            veto_events += 1
            pbar.update(n=1)
            continue

        
       	str_trace = f"Trace_[{i}]"
        trace = np.array(h5file[str_trace][:])
        max_val = np.argmax(trace)
        low_bound = max_val - 75
        if low_bound < 0:
            low_bound = 5
        upper_bound = max_val + 75
        if upper_bound > 511:
            upper_bound = 506
        trace = trace[low_bound:upper_bound]

        polynomial_degree=2 
        baseObj=BaselineRemoval(trace)
        trace=baseObj.IModPoly(polynomial_degree)

        #if np.sum(trace) > 800000:
        #    veto_events += 1
        #    pbar.update(n=1)
        #    continue

        len_list.append(length)
        width_list.append(width)
        depth_list.append(depth)
        tot_energy.append(np.sum(trace))
        tracemax_list.append(np.max(trace))
        tracedev_list.append(np.std(trace))
        pads_list.append(len(trace[trace > 0]))

        # Track event number of good events
        good_events.append(i)  
        pbar.update(n=1)

    return (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list)

In [7]:
# assign directory
directory = "C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/"

output_df = pd.DataFrame(columns=['file','event','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

file_names = []
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if f[-3:] == '.h5':
            print(f)
            file_names.append(filename)
            h5f = h5py.File(directory+filename, 'r')
            (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list) = main(h5file=h5f, threshold=15)
            for event in range(len(tot_energy)):
                output_df = output_df.append({'file' : filename, 
                                              'event' : event, 
                                              'length' : len_list[event], 
                                              'width' : width_list[event], 
                                              'depth' : depth_list[event], 
                                              'tracesum' : tot_energy[event], 
                                              'tracemax' : tracemax_list[event], 
                                              'tracedev' : tracedev_list[event], 
                                              'padnum' : pads_list[event]}
                                             ,ignore_index = True)


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/273_800p.h5


100%|█████████▉| 676/677 [00:13<00:00, 51.64it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning3protons0-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 40.12it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning3protons1-800p.h5


...

 98%|█████████▊| 50/51 [00:01<00:00, 42.23it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning3protons98-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 43.65it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning3protons99-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 48.70it/s]


In [8]:
for file in file_names:
    print(file)
    print('length:\t', output_df[output_df['file'] == file]['length'].mean(),'\t', output_df[output_df['file'] == file]['length'].std())
    print('width:\t', output_df[output_df['file'] == file]['width'].mean(),'\t', output_df[output_df['file'] == file]['width'].std())
    print('depth:\t', output_df[output_df['file'] == file]['depth'].mean(),'\t', output_df[output_df['file'] == file]['depth'].std())
    print('tracesum:\t', output_df[output_df['file'] == file]['tracesum'].mean(),'\t', output_df[output_df['file'] == file]['tracesum'].std())
    print('tracemax:\t', output_df[output_df['file'] == file]['tracemax'].mean(),'\t', output_df[output_df['file'] == file]['tracemax'].std())
    print('tracedev:\t', output_df[output_df['file'] == file]['tracedev'].mean(),'\t', output_df[output_df['file'] == file]['tracedev'].std())
    print('padnum:\t', output_df[output_df['file'] == file]['padnum'].mean(),'\t', output_df[output_df['file'] == file]['padnum'].std())
    print()

273_800p.h5
length:	 13.598839169264545 	 0.36378981766204926
width:	 5.490411630687277 	 1.079046393352792
depth:	 1.1956686193850883 	 0.4627512586642447
tracesum:	 155282.26997909456 	 1353.2815542925161
tracemax:	 17261.59640386621 	 3820.7164201418886
tracedev:	 3396.80817140803 	 395.6029936835511
padnum:	 89.44822485207101 	 4.124217163844774

tuning3protons0-800p.h5
length:	 14.592615707306718 	 1.1526767462669063
width:	 8.573641052536388 	 0.9034343261255041
depth:	 3.644100760765216 	 0.9289545240259964
tracesum:	 163846.138662618 	 1411.6020089156248
tracemax:	 16022.70730437251 	 2124.4567464204424
tracedev:	 3352.98476602642 	 250.05157110139254
padnum:	 105.96 	 11.000853399177906

tuning3protons1-800p.h5
length:	 13.819244723975151 	 1.0751786209191898
width:	 7.512977382597627 	 0.8717554379353093
depth:	 3.258058805853486 	 0.8211319399307213
tracesum:	 128339.49734028534 	 1198.7855607685233
tracemax:	 12822.224538464357 	 1778.0367853583941
tracedev:	 2655.393947332

In [9]:
output_df.to_csv('event summary.csv', index=False)

In [10]:
output_df = pd.read_csv('event summary.csv')
file_names = output_df['file'].unique() 

run_df = pd.DataFrame(columns=['file','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

reference_run = '273_800p.h5'

for file in file_names:
    run_df = run_df.append({'file' : file, 
                            'length' : output_df[output_df['file'] == file]['length'].mean(), 
                            'width' : output_df[output_df['file'] == file]['width'].mean(), 
                            'depth' : output_df[output_df['file'] == file]['depth'].mean(), 
                            'tracesum' : output_df[output_df['file'] == file]['tracesum'].mean(), 
                            'tracemax' : output_df[output_df['file'] == file]['tracemax'].mean(), 
                            'tracedev' : output_df[output_df['file'] == file]['tracedev'].mean(), 
                            'padnum' : output_df[output_df['file'] == file]['padnum'].mean()}
                           ,ignore_index = True)

run_df['length'] = (run_df['length'] - output_df[output_df['file'] == reference_run]['length'].mean()) / output_df[output_df['file'] == reference_run]['length'].std()
run_df['width'] = (run_df['width'] - output_df[output_df['file'] == reference_run]['width'].mean()) / output_df[output_df['file'] == reference_run]['width'].std()
run_df['depth'] = (run_df['depth'] - output_df[output_df['file'] == reference_run]['depth'].mean()) / output_df[output_df['file'] == reference_run]['depth'].std()
run_df['tracesum'] = (run_df['tracesum'] - output_df[output_df['file'] == reference_run]['tracesum'].mean()) / output_df[output_df['file'] == reference_run]['tracesum'].std()
run_df['tracemax'] = (run_df['tracemax'] - output_df[output_df['file'] == reference_run]['tracemax'].mean()) / output_df[output_df['file'] == reference_run]['tracemax'].std()
run_df['tracedev'] = (run_df['tracedev'] - output_df[output_df['file'] == reference_run]['tracedev'].mean()) / output_df[output_df['file'] == reference_run]['tracedev'].std()
run_df['padnum'] = (run_df['padnum'] - output_df[output_df['file'] == reference_run]['padnum'].mean()) / output_df[output_df['file'] == reference_run]['padnum'].std()

# sum of absolute deviations from reference run
run_df['Deviation'] = abs(run_df['length']) + abs(run_df['width']) + abs(run_df['depth']) + abs(run_df['tracesum']) + abs(run_df['tracemax']) + abs(run_df['tracedev']) + abs(run_df['padnum'])

In [11]:
run_df = run_df.sort_values(by=['Deviation'])
best_run_df = run_df.head(10)

# load in parameters for best runs
params = pd.read_csv('C:\\Users\\Adam\\OneDrive - Tenundra, Inc\\Desktop\\Programming\\GADGET2\\simInput\\parameters.csv')

best_params = params[params['Sim'].isin(best_run_df['file'].str.replace('.h5',''))]

In [12]:
parameters = ['Threshold', 'EIonize', 'Fano', 'CoefL', 'CoefT', 'Gain', 'GETGain', 'PeakingTime']
print('Current Best Fit Parameters')
for p in parameters:
    print(p, '\t', best_params[p].mean(),'\t', best_params[p].std()/best_params[p].mean(), '\n', params[p].mean(),'\t', params[p].std()/params[p].mean(), '\n')

Current Best Fit Parameters
Threshold 	 23.77777777777778 	 0.09351633689808177 
 23.99 	 0.09975619015210048 

EIonize 	 34.077777777777776 	 0.06723345128459007 
 30.81299999999999 	 0.08923407261919866 

Fano 	 0.22999999999999998 	 0.06874516652539957 
 0.21700000000000003 	 0.09813995157555704 

CoefL 	 0.0001231111111111111 	 0.13691368461845344 
 0.00011979000000000002 	 0.09952174722736559 

CoefT 	 0.0021355555555555555 	 0.07437528184632335 
 0.0020613 	 0.09635408327843778 

Gain 	 7355.188888888889 	 0.07137807121293253 
 8514.405999999999 	 0.10586381636347995 

GETGain 	 139.54444444444445 	 0.09357287956422802 
 133.583 	 0.09717994063943178 

PeakingTime 	 560.8888888888889 	 0.08125554875932296 
 581.89 	 0.09331425331027843 



In [13]:
print('Full Run Deviation:', run_df['Deviation'].mean(), '\t', run_df['Deviation'].std()/run_df['Deviation'].mean())
print('Best Run Deviation:', best_run_df['Deviation'].mean(), '\t', best_run_df['Deviation'].std()/best_run_df['Deviation'].mean())

Full Run Deviation: 73.20810689912018 	 0.501081506857744
Best Run Deviation: 19.611030850182193 	 0.4159933817008462


In [14]:
best_run_df.head()

,file,length,width,depth,tracesum,tracemax,tracedev,padnum,Deviation
0,273_800p.h5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,tuning3protons11-800p.h5,1.289383,2.184066,5.474724,0.545206,-0.431869,-0.428415,2.461504,12.815166
43,tuning3protons47-800p.h5,1.581615,2.241685,5.160513,3.108044,-0.638056,-0.570498,2.985239,16.285652
71,tuning3protons72-800p.h5,2.262888,2.620358,5.062441,-5.940334,-0.363693,-0.550198,2.990089,19.790001
1,tuning3protons0-800p.h5,2.731733,2.857365,5.291033,6.328224,-0.324256,-0.110776,4.003614,21.647001
